# Workspace for Data Compression Tasks
[Click here for the source repository](https://github.com/MarkGotham/Data_Compression)

In [2]:
from IPython.display import Markdown
Markdown("./tasks/explore_noise.md")

## Background

First [`composite`](https://github.com/MarkGotham/Data_Compression/blob/main/composite.ipynb)
created a roundtrip with a composite signal and Fourier transform on it.
Then [`masking`](https://github.com/MarkGotham/Data_Compression/blob/main/masking.ipynb)
explored the effect of noise masking a sound in _theory_.
This notebook provides a space for experimenting with combined sound and noise signals in _practice_.


## Task

- Type: Explore/experiment
- Task:
  - Play around with the frequency and amplitude of
    - the periodic signal (and or composite signal),
    - the noise (in which case, by filtering the frequency _range_)
  - Note your limits: when do you stop hearing the signal?
- Bonus tasks:
  - Go to the `masking` functions and see how well they match your experience.
  - Write a function to filter the noise, to clear the noise spectrum around a signal periodicity. 
  - Plot Fourier transforms of the white noise and the filtered noise to compare.


## Exploration Template

In [ ]:
from IPython.display import Audio
import numpy as np

In [ ]:
import numpy as np
sr = 22050 # sample rate
# sr *= 8
T = 1.0    # seconds
hz = 440
time_array = np.linspace(0, T, int(T * sr))

### Signal

In [ ]:
# One tone alone

amp = 1/2
multiplier = 1/2
one_tone = amp * np.sin(2 * np.pi * multiplier * hz * time_array)
Audio(one_tone, rate=sr)

In [ ]:
# # Composite signal (optional)

# from implementations import composite_signal
# comp = composite_signal.composite_signal(
#     time_array,
#     overtone_magnitude_dict = composite_signal.fundamental_w_partials(multiples = range(1, 50))
# )
# Audio(comp, rate=sr)

## Noise

### White Noise alone

In [ ]:
white_noise = np.random.normal(0, 1, int(T * sr))
Audio(white_noise, rate=sr)

### Filtered noise
With thanks to [scipy-cookbook](https://scipy-cookbook.readthedocs.io/items/ButterworthBandpass.html)

In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
# Sample rate and desired cutoff frequencies (in Hz).
fs = 5000.0
lowcut = 50.0
highcut = 1550.0

filter_noise = butter_bandpass_filter(white_noise, lowcut, highcut, fs, order=6)

Audio(filter_noise, rate=sr)

## Combinations

In [ ]:
Audio(one_tone + filter_noise, rate=sr)

In [ ]:
Audio(one_tone + white_noise, rate=sr)

## Plots to visualise

What does this filtering do? Let's compare Fourier transforms of the white noise and the filtered noise profiles.

In [ ]:
import matplotlib.pyplot as plt

def plot_fourier(noise_profile: np.array = white_noise):
    X = np.fft.fft(noise_profile)
    X_mag = np.absolute(X)
    f = np.linspace(0, sr, len(X_mag))
    
    plt.figure(figsize=(10, 6))
    
    plot_proportion = 1  # how much to plot
    plt.plot(f[:int(len(f) * plot_proportion)], X_mag[:int(len(X_mag) * plot_proportion)])
    
    axis_label_size = 14
    plt.ylabel('FFT Amplitude $|X(freq)|$', fontsize=axis_label_size)
    plt.xlabel('Frequency, $Hz$', fontsize=axis_label_size)

In [ ]:
plot_fourier(noise_profile = white_noise)

In [ ]:
plot_fourier(noise_profile = filter_noise)